In [ ]:
import torch

In [ ]:
#exetute this cell only in Google Colab
!pip install transformers
!pip install datasets

In [ ]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from datasets import load_metric
from sklearn.model_selection import train_test_split

In [ ]:
class RMPDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
#exetute this cell only in Google Colab
rmp = pd.read_csv("rmp_small.csv")

In [ ]:
rmp

,review,score
0,"he can be a mean man sometimes, but just get o...",2
1,"He's fun, but he speeds through lectures and l...",2
2,Dr. Foster is an amazing teacher. One of my fa...,4
3,She is one of the greatest teacher I have ever...,4
4,Professor Bird was very easy and relaxed. If y...,3
...,...,...
97540,Charismatic and enthusiastic but that's it . H...,0
97541,"Great books - Dickens, Eliot, etc. A little d...",3
97542,Awesome professor. Doesnt stop explaining unti...,4
97543,"He has a really high standard, but grades fair...",4


In [ ]:
sentences = list(rmp['review'])
labels = list(rmp['score'])

In [ ]:
#remove nan from both lists
selnan = list()
for idx,name in enumerate(sentences):
    if name == '' or pd.isnull(name):
        selnan.append(idx)
sentences = [sentences[i] for i, e in enumerate(sentences) if i not in selnan]
labels = [labels[i] for i, e in enumerate(labels) if i not in selnan]
(len(sentences), len(labels))

(97545, 97545)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.05, random_state=42, shuffle=True)

In [ ]:
checkpoint = "microsoft/MiniLM-L12-H384-uncased"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
train_encodings = tokenizer(X_train, truncation=True, padding=True)
#val_encodings = tokenizer(X_val, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpqr02hrd_


Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

storing https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/29039dfe8c131360348e9f5ebecd464478cec7576c9af532b55ddcf9d4ec8d1e.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b
creating metadata file for /root/.cache/huggingface/transformers/29039dfe8c131360348e9f5ebecd464478cec7576c9af532b55ddcf9d4ec8d1e.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b
https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpym2u0o9j


Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

storing https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/ceb753d3f27a8c0d09184f35884666cda91b8ae610cd2a54d89793ac7663f1f9.13815020fd994b27db9974c0ce0ec4c47dfac6c8f11bf1a35a0a06d5b165665a
creating metadata file for /root/.cache/huggingface/transformers/ceb753d3f27a8c0d09184f35884666cda91b8ae610cd2a54d89793ac7663f1f9.13815020fd994b27db9974c0ce0ec4c47dfac6c8f11bf1a35a0a06d5b165665a
loading configuration file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ceb753d3f27a8c0d09184f35884666cda91b8ae610cd2a54d89793ac7663f1f9.13815020fd994b27db9974c0ce0ec4c47dfac6c8f11bf1a35a0a06d5b165665a
Model config BertConfig {
  "_name_or_path": "microsoft/MiniLM-L12-H384-uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_ra

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

storing https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/49c302ee103bf6737d0877cfbd658563cf4bbc4b7914363ca419ce8a3d8a4c51.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/49c302ee103bf6737d0877cfbd658563cf4bbc4b7914363ca419ce8a3d8a4c51.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpvof115zu


Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

storing https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/1e5909e4dfaa904617797ed35a6105a23daa56cbefca48fef329f772584699fb.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /root/.cache/huggingface/transformers/1e5909e4dfaa904617797ed35a6105a23daa56cbefca48fef329f772584699fb.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/49c302ee103bf6737d0877cfbd658563cf4bbc4b7914363ca419ce8a3d8a4c51.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/added_tokens.json from cache at None
loading fil

In [ ]:
train_dataset = RMPDataset(train_encodings, y_train)
#val_dataset = RMPDataset(val_encodings, y_val)
test_dataset = RMPDataset(test_encodings, y_test)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
training_args = TrainingArguments(
    "RMP_5_labels",
    per_device_train_batch_size=32,# batch size per device during training
    evaluation_strategy="no",
    save_strategy="no",
    num_train_epochs=3
 )

training_args

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_metrics=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=5)

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_dataset,
#    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

loading configuration file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ceb753d3f27a8c0d09184f35884666cda91b8ae610cd2a54d89793ac7663f1f9.13815020fd994b27db9974c0ce0ec4c47dfac6c8f11bf1a35a0a06d5b165665a
Model config BertConfig {
  "_name_or_path": "microsoft/MiniLM-L12-H384-uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "ab

Downloading:   0%|          | 0.00/127M [00:00<?, ?B/s]

storing https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/b774244369e464de2c660477b70bae7c3223fa7250aa1c8fc0b0f037ed58418a.087808d17814e241e9352c5ce0fea1a7d05e5b0f020d44b42b5f05922e96c923
creating metadata file for /root/.cache/huggingface/transformers/b774244369e464de2c660477b70bae7c3223fa7250aa1c8fc0b0f037ed58418a.087808d17814e241e9352c5ce0fea1a7d05e5b0f020d44b42b5f05922e96c923
loading weights file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/b774244369e464de2c660477b70bae7c3223fa7250aa1c8fc0b0f037ed58418a.087808d17814e241e9352c5ce0fea1a7d05e5b0f020d44b42b5f05922e96c923
All model checkpoint weights were used when initializing BertForSequenceClassification.

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initi

In [ ]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 92667
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8688


Step,Training Loss
500,1.148900
1000,1.019400
1500,1.004700
2000,0.984300
2500,0.978400
3000,0.962800
3500,0.923600
4000,0.930500
4500,0.922100
5000,0.918400




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=8688, training_loss=0.9345109124649076, metrics={'train_runtime': 2453.5928, 'train_samples_per_second': 113.304, 'train_steps_per_second': 3.541, 'total_flos': 5258026702493082.0, 'train_loss': 0.9345109124649076, 'epoch': 3.0})

In [36]:
predictions = trainer.predict(test_dataset)
print(predictions.predictions.shape, predictions.label_ids.shape)
predictions.predictions[:10]
predictions.label_ids[:10]

compute_metrics([predictions.predictions, predictions.label_ids])

***** Running Prediction *****
  Num examples = 4878
  Batch size = 8


(4878, 5) (4878,)


{'accuracy': 0.575850758507585}

In [37]:
data = [ pd.Series(prd) for prd in predictions.predictions]
y_pred = [prd.idxmax() for prd in data]
cfm = pd.crosstab(np.array(y_test), np.array(y_pred), rownames=['Actual'], colnames=['Predicted'])
cfm = cfm / cfm.sum().sum()

In [38]:
cfm

Predicted,0,1,2,3,4
Actual,,,,,
0,0.123411,0.02009,0.006560,0.002050,0.008815
1,0.038950,0.03280,0.023780,0.007790,0.007380
2,0.011480,0.02214,0.035670,0.037310,0.018040
3,0.003280,0.00410,0.025215,0.094096,0.122181
4,0.000820,0.00082,0.008200,0.055146,0.289873
